In [12]:
import json
import ollama
import numpy as np

class RRT:
    def __init__(self, model, goal, branches, rounds):
        self.client = ollama
        self.model = model
        self.goal = goal
        self.n = branches
        self.r = rounds

    def multi_turn_conversation(self, rounds):
        messages = []

        user_input = "Now, imagine what happens if you go straight further?"
        scene_descriptions = []

        for _ in range(rounds):  
            messages.append({"role": "user", "content": user_input})

            response = self.client.chat(
                model=self.model,
                messages=messages
            )
            response_text = response['message']['content']
            
            scene_descriptions.append(response_text)
            messages.append({"role": "assistant", "content": response_text})

        print(scene_descriptions)
        return scene_descriptions

    def multi_turn_conversation_ollama(self, model, initial_description, rounds):
        messages = [
            {"role": "system", "content": f"You are a navigator tasked with outputting a new scene description conditioned on the description given with your navigation commonsense of what possibly ahead. Description: '{initial_description}'"},
        ]

        user_input = "Now, imagine what happens if you go straight further?"
        scene_descriptions = []

        for _ in range(rounds):  
            messages.append({"role": "user", "content": user_input})
              
            response = self.client.chat(
                model=model,
                messages=messages
            )
            print(response)
            ollama_response_text = response['message']['content']
            scene_descriptions.append(ollama_response_text)
            messages.append({"role": "assistant", "content": ollama_response_text})

        return scene_descriptions

    def generate_scene_descriptions(self, initial_description):
        branches_descriptions = []
        for _ in range(self.n):
            scene_descriptions = self.multi_turn_conversation_ollama(self.model, initial_description, self.r)
            branches_descriptions.append(scene_descriptions)
        return branches_descriptions

    def score_scene_description_ollama(self, scene_description):
        response = self.client.chat(
            model=self.model,
            messages=[
                {"role": "system", "content": f"How likely is this scene related to the goal: '{self.goal}'? Scene: '{scene_description}'"},
                {"role": "user", "content": "Score the relevance on a scale of 1 to 5. Output the score in this format: <<<score>>>"},
            ]
        )
        score_text = response['message']['content']
        try:
            score = float(score_text)
            print("!!!!!", score)
        except ValueError:
            print("Error")
            score = np.random.choice([1,2,3,4,5])
        return score
    
    def score_scene_description(self, scene_description):
        messages = [
            {"role": "user", "content": f"Generate a score randomly from 1 to 5."},
        ]

        response = self.client.chat(
            model=self.model,
            messages=messages
        )
        response_text = response['message']['content']
        print("RESPONSE: ", response_text, "END")
        try:
            score_text = response_text.split('<<<')[1].split(">>>")[0].strip()
            print("score_text: ", score_text)
            score = float(score_text)
        except Exception:
            print("Error with scoring!")
            score = np.random.choice([1,2,3,4,5])
        return score

    def run_rrt(self, initial_description):
        branches_descriptions = self.generate_scene_descriptions(initial_description)
        
        branch_scores = []
        for branch in branches_descriptions:
            branch_score = sum([self.score_scene_description(desc) for desc in branch]) / len(branch)
            branch_scores.append(branch_score)
        
        tree_score = sum(branch_scores) / len(branch_scores)
        return tree_score, branches_descriptions


In [14]:
LLT = RRT('llama3', 'car', 3, 3)
LLT.run_rrt('Now im at the parking lot.')

{'model': 'llama3', 'created_at': '2024-07-11T04:20:41.505332Z', 'message': {'role': 'assistant', 'content': "If I continue going straight from the parking lot, I would expect to see a building or a structure of some kind ahead. Perhaps it's the entrance to a shopping mall, an office complex, or even a hotel.\n\nAs I walk forward, the pavement beneath my feet becomes more defined, and the sounds of cars and chatter fade away, replaced by the hum of air conditioning units and the murmur of people moving about. The building's facade comes into view, its glass and steel exterior reflecting the sunlight in dazzling patterns.\n\nI can imagine that if I step inside, I'll find myself in a lobby with marble floors, sleek furniture, and possibly even a reception desk or two. The air will be cool and clean, with a faint scent of fresh coffee wafting from the café area.\n\nOf course, this is just one possible scenario – there could be many other paths forward, but based on my navigation commonsen

(2.444444444444444,
 [["If I continue going straight from the parking lot, I would expect to see a building or a structure of some kind ahead. Perhaps it's the entrance to a shopping mall, an office complex, or even a hotel.\n\nAs I walk forward, the pavement beneath my feet becomes more defined, and the sounds of cars and chatter fade away, replaced by the hum of air conditioning units and the murmur of people moving about. The building's facade comes into view, its glass and steel exterior reflecting the sunlight in dazzling patterns.\n\nI can imagine that if I step inside, I'll find myself in a lobby with marble floors, sleek furniture, and possibly even a reception desk or two. The air will be cool and clean, with a faint scent of fresh coffee wafting from the café area.\n\nOf course, this is just one possible scenario – there could be many other paths forward, but based on my navigation commonsense, this seems like a likely outcome!",
   "If I continue going straight from where I 

In [ ]:
from RRT